# Утилиты

In [61]:
from sympy.solvers import solve
from sympy import Symbol
import numpy as np
from typing import Tuple, Callable, List, Dict
import plotly.express as px
import plotly.graph_objects as go
from scipy.special import gamma
import pandas as pd
from tqdm import tqdm

In [62]:
Original = Callable[[int], complex]
Image = Callable[[complex], complex]

tqdm.pandas()

In [63]:
def get_optimal_parameters(n: int, M: float = 2, eps: float = 10 ** -16) -> Tuple[float, int]:
    coeff = [1, -1] + [0 for _ in range(n - 1)] + [10 ** -4]
    roots = np.roots(coeff)
    real_roots = roots[np.isreal(roots)].real
    r = real_roots[real_roots >= 0].min()

    m = n + 1
    error = (M * r ** m) / (1 - r ** m) * 10 ** 12
    while error > 1:
        m += 1
        error = (M * r ** m) / (1 - r ** m) * 10 ** 12

    return r, m

In [64]:
def get_w(F: Image, n: int, M: float = 2) -> Original:
    phi = lambda p: p * F(p)
    r, m = get_optimal_parameters(n, M)

    e = lambda x: np.exp(1j * 2 * np.pi * x / m)
    re = lambda x: r * e(x)

    return lambda t: 1 / m * sum([re(j) ** (-n) / (1 - re(j)) * phi(n / t * (1 - re(j))) for j in range(1, m + 1)])

In [65]:
def find_coefficients(numbers: List[float]) -> List[float]:
    coefficients = []
    for number_i in numbers:
        coefficient = 1
        for number in numbers:
            if number == number_i:
                continue
            coefficient *= number_i / (number_i - number)
        coefficients.append(coefficient)
    return coefficients


# def find_optimal_numbers(k: int, n_1: int, n_k: int) -> List[Tuple[float, int]]:
#     roots = [1 / 2 * ((1 / n_1 - 1 / n_k) * np.cos(j * np.pi / (k - 1)) + 1 / n_1 + 1 / n_k) for j in range(k)]
#     numbers = [roots[0] / root for root in roots]

#     return list(zip(find_coefficients(numbers), [round(1 / root + 1 / 2) for root in roots]))

In [66]:
def get_optimized_w_values(w_values: List[np.ndarray], numbers: List[int]) -> np.ndarray:
    coefficients = find_coefficients(numbers)
    return sum(coefficient * w_value for coefficient, w_value in zip(coefficients, w_values))

In [67]:
def banchmark_slow(
    n: int,
    image: Image,
    original_values: List[float],
    linspace: np.linspace
) -> pd.Series:
    w = np.vectorize(get_w(image, n))
    
    approx_values = w(linspace).real
    error = abs(approx_values - original_values).max()

    print(f'{n}: {error}')

    return pd.Series([error, list(approx_values)])

In [68]:
def banchmark_fast(
    n: int,
    original_values: List[float],
    data_slow: pd.DataFrame,
    k: int = 5,
) -> pd.Series:
    last = (n - k if n > k else 0) + 1
    numbers = list(range(last, n + 1))

    w_values = []
    data_slow[data_slow.n.isin(numbers)].apply(lambda row: w_values.append(np.array(eval(str(row.w)))), axis=1)

    approx_values = get_optimized_w_values(w_values, numbers)

    error = abs(approx_values - original_values).max()
    print(f'{n}: {error}')

    return pd.Series([error, list(approx_values)])

In [84]:
def plot_errors(data: Dict[str, pd.DataFrame], dtick: float = 0.25) -> go.Figure:
    fig = go.Figure()

    for name, df in data.items():
        fig.add_scatter(x=df.n, y=df.error, name=name)

    fig.update_layout(title='Зависимость ошибки от n')
    fig.update_xaxes(title='n')
    fig.update_yaxes(title='Ошибка', tickmode = 'linear', tick0 = 0, dtick = dtick, tickformat='.2f', type='log')

    return fig

In [70]:
def plot_w(data: pd.DataFrame, original_values: List[float], linspace: np.linspace, k: int = 5) -> go.Figure:
    filtered_data = pd.concat([data.head(k), data.tail(k)])

    fig = go.Figure()
    filtered_data.apply(lambda row: fig.add_scatter(x=linspace, y=eval(str(row.w)), name=f'{row.n}'), axis=1)
    fig.add_scatter(x=linspace, y=original_values, name='Оригинал', marker_color='black')

    fig.update_layout(title='Полученные приближения для разных n', legend_title='n')
    fig.update_xaxes(title='x')
    fig.update_yaxes(title='y')

    return fig

# Sheet 2

In [71]:
F = lambda p: (np.sqrt(p) + 1) / (p ** 2 + 1)
# F = lambda p: 1 / (p + 1)

segment = (10 ** -12, 10)
N = 10000

t = np.linspace(*segment, N)

SERIES_LIMIT = 40

In [72]:
original = lambda t: sum((-1) ** k / gamma(2 * k + 3 / 2) * t ** (2 * k + 1 / 2) for k in range(SERIES_LIMIT)) + np.sin(t)
original = np.vectorize(original)

original_values = original(t).real

In [73]:
fig = px.line(x=t, y=original_values)

fig.update_layout(title=f'Сумма первых {SERIES_LIMIT} членов ряда', template=None, margin={'l': 50, 'r': 50, 't': 50, 'b': 50})

fig.show()

fig.write_image("original.svg")

Unsupported

In [74]:
for n in [*range(1, 6), *range(46, 51)]:
    r, m = get_optimal_parameters(n)
    print(f'{n}: r={round(r, 6)}, {m=}')

1: r=0.0001, m=4
2: r=0.010051, m=7
3: r=0.04717, m=10
4: r=0.102748, m=13
5: r=0.164281, m=16
46: r=0.853441, m=179
47: r=0.856739, m=184
48: r=0.859903, m=188
49: r=0.862942, m=193
50: r=0.865863, m=197


In [75]:
# data_slow = pd.DataFrame(range(1, 51), columns=['n'])
# data_slow[['error', 'w']] = data_slow.progress_apply(lambda row: banchmark_slow(row.n, F, original_values, t), axis=1)
# data_slow.to_csv('data_slow.csv', index=False)
# data_slow

In [76]:
data_slow = pd.read_csv('data_slow.csv')
data_slow

,n,error,w
0,1,1.855477,"[1.5000019999997574e-06, 0.04943662212598099, ..."
1,2,1.878187,"[1.3258267147257558e-06, 0.04342847580263001, ..."
2,3,1.880964,"[1.2629550471857447e-06, 0.041273536871437956,..."
3,4,1.869177,"[1.230470000000692e-06, 0.04016288538047355, 0..."
4,5,1.848781,"[1.2106223784427521e-06, 0.03948521705086172, ..."
5,6,1.823244,"[1.1972371200488098e-06, 0.03902858332487151, ..."
6,7,1.794573,"[1.1875994747973037e-06, 0.0386999892441595, 0..."
7,8,1.763970,"[1.1803286751697775e-06, 0.03845219752902442, ..."
8,9,1.732193,"[1.1746481244589297e-06, 0.0382586646891743, 0..."
9,10,1.699750,"[1.1700875028012915e-06, 0.03810332668537127, ..."


In [88]:
fig = plot_errors({'Без ускорения': data_slow}, dtick=0.03)

fig.update_layout(template=None, margin={'l': 100, 'r': 50, 't': 50, 'b': 50})
fig.write_image('slow_error.svg')

fig.show()

Unsupported

In [78]:
fig = plot_w(data_slow, original_values, t)

fig.update_layout(template=None, margin={'l': 50, 'r': 50, 't': 50, 'b': 50})
fig.write_image('slow_w.svg')

fig.show()

Unsupported

In [79]:
data_fast = pd.DataFrame(range(1, 51), columns=['n'])
data_fast[['error', 'w']] = data_fast.progress_apply(lambda row: banchmark_fast(row.n, original_values, data_slow, 5), axis=1)
data_fast.to_csv('data_fast.csv', index=False)
data_fast

1: 1.8554766984602202
2: 1.9040010266183094
3: 1.8779767149493962
4: 1.7488171231399032
5: 1.6092892773819474
6: 1.5531720521997314
7: 1.4555767181327075
8: 1.3369041073804757
9: 1.2083845676279066
10: 1.0763985447643405
11: 0.9451794636525191
12: 0.8180205726155654
13: 0.6976391510263298
14: 0.5861790997567695
15: 0.48513412070570183
16: 0.3966129631331673
17: 0.35157038115791617
18: 0.31209028866462574
19: 0.27747453863464444
20: 0.24710358028805984
21: 0.22043291288792383
22: 0.19698406640122784
23: 0.17634199910285941
24: 0.1581435269437516
25: 0.14207561262084645
26: 0.1278657434167093
27: 0.1152781572557629
28: 0.1041090386269018
29: 0.0941818588432849
30: 0.08534336152539657
31: 0.07745983321915506
32: 0.07041761358062193
33: 0.06411592768606422
34: 0.05846647455809462
35: 0.05339494796468158
36: 0.04883442198306376
37: 0.044724641172336765
38: 0.04102099776107393
39: 0.03767100526287914
40: 0.03464360559651114
41: 0.03189667881580438
42: 0.029405807843588927
43: 0.0271430565266

100%|██████████| 50/50 [00:06<00:00,  7.86it/s]


,n,error,w
0,1,1.855477,"[1.5000019999997574e-06, 0.04943662212598099, ..."
1,2,1.904001,"[1.1516514294517542e-06, 0.037420329479279035,..."
2,3,1.877977,"[1.1299918534327061e-06, 0.03673532377394126, ..."
3,4,1.748817,"[1.1284267219028913e-06, 0.036685829150162, 0...."
4,5,1.609289,"[1.128372988159791e-06, 0.03668412989417391, 0..."
5,6,1.553172,"[1.1283780677029376e-06, 0.036684290532612795,..."
6,7,1.455577,"[1.128379443206422e-06, 0.03668433403358495, 0..."
7,8,1.336904,"[1.1283798722893116e-06, 0.03668434760099615, ..."
8,9,1.208385,"[1.1283800298771758e-06, 0.036684352589707814,..."
9,10,1.076399,"[1.128380096465458e-06, 0.03668435468551223, 0..."


In [80]:
fig = plot_errors({'С ускорением': data_fast})

fig.update_layout(template=None, margin={'l': 100, 'r': 50, 't': 50, 'b': 50})
fig.write_image('fast_error.svg')

fig.show()

Unsupported

In [81]:
fig = plot_w(data_fast, original_values, t)

fig.update_layout(template=None, margin={'l': 50, 'r': 50, 't': 50, 'b': 50})
fig.write_image('fast_w.svg')

fig.show()

Unsupported

In [82]:
fig = plot_errors({'Без ускорения': data_slow, 'С ускорением': data_fast})

fig.update_layout(template=None, margin={'l': 100, 'r': 50, 't': 50, 'b': 50})
fig.write_image('cmp_error.svg')

fig.show()

Unsupported